# 데이터

### 데이터셋 불러오기

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import pandas as pd
data = pd.read_json('../data/raw_data/Web/경기도외국인지원센터_상담사례.json')
data.head()

,제목,작성일,상담유형,거주지역,국적,체류자격,상담내용,진행 과정 및 결과,관련법령 및 정보,평가 및 의의,상담지원단체,작성자,url
0,중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원,23-11-27,체류자격,여주시,우즈베키스탄,E-9,중도 퇴직하는 외국인 근로자의 추가 납부세액 사례,2023-09-18\n- 고용노동부에서 외국인근로자의 2020년 종합소득세 체납을 ...,,,여주시외국인복지센터,이수현,https://gmhr.or.kr/case/1703?page=1
1,국민연금 반환일시금 신청 지원,23-11-27,기타,남양주시,필리핀,미등록,2023년 8월 중순 미등록체류 상황에서 본국으로 강제출국 당한 필리핀 국적의 이주...,11월 2일\n1. 노동자로부터 우편으로 관련서류 수령.\n 1) 외국인등록증 사본...,,,남양주시외국인복지센터,김성수,https://gmhr.or.kr/case/1702?page=1
2,외국인아동 교육권 보장을 위한 체류자격 부여자(G-1-85) 외국인등록 절차 지원,23-11-27,체류자격,남양주시,필리핀,미등록,법무부 장기 미등록체류 이주아동 구제대책으로 23년 4월에 접수한 필리핀 국적의 아...,11월 1일\n1. 양주출입국 사범과 방문.\n2. 담당자로부터 유전자 검사 결과 ...,,,남양주시외국인복지센터,김성수,https://gmhr.or.kr/case/1701?page=1
3,단체행동을 통해 체불임금 해결한 사례,23-11-27,임금체불,안산시,필리핀,E-9,"사례자 안산시 한 직장에서 근무하는데, 회사에서 잔업수당 등을 상습적으로 속이고 있...",8월 14일\n(전화상담) 회사에서 급여계산을 잘못해서 주고 야간 수당을 안주고 있...,,,글로벌미션센터,최경식,https://gmhr.or.kr/case/1700?page=1
4,취업가능기간이 짧게 남아 취업이 어려운 노동자 구직활동 지원,23-11-27,고용허가,시흥시,필리핀,E-9,상기 사례자는 한국 체류기간(노동가능기간)이 1년 4개월로 비교적 짧게 남아서 면접...,4월 2일\n(내방상담) 현재 다니는 2번째 회사에도 일이 없어서 사업장변경 신청하...,,,글로벌미션센터,최경식,https://gmhr.or.kr/case/1699?page=1


In [2]:
data.shape

(1700, 13)

### 2023년 이후의 사례만

In [3]:
# 작성일 열을 datetime으로 변환
data['작성일'] = pd.to_datetime(data['작성일'], format='%y-%m-%d')

In [4]:
from datetime import datetime

# 2023년 1월 1일 이후의 데이터만 필터링
cutoff_date = datetime(2023, 1, 1)
filtered_data = data[data['작성일'] >= cutoff_date]

# 데이터셋 생성

저장 형식:



Document (
- 상담제목(dict) : raw_text (str), Embedding(list)
- 상담내용요약(dict) : raw_text(str), Embedding(list)
- 진행 과정 및 결과(str)
- 내담자 정보(dict) : {
거주지역(str),
국적(str),
체류자격(str),
추가정보 (list): (LLM에서 추출)
}
- 해결방법 (str) : (LLM에서 추출)
- metadata (dict) : {
상담유형 (str),
상담일시 (str)
}


## 포멧 변환

제목, 내용 요약(상담내용), 진행과정 및 결과, 내담자정보, 상담유형, 상담일시

In [ ]:
import pandas as pd
from typing import Dict, List
import json

def transform_dataframe_to_documents(df: pd.DataFrame) -> List[Dict]:
    """
    데이터프레임을 지정된 문서 형식으로 변환

    Args:
        df: 입력 데이터프레임

    Returns:
        변환된 문서 리스트
    """
    documents = []

    for _, row in df.iterrows():
        document = {
            "상담제목": {
                "raw_text": row["제목"],
                "Embedding": []
            },
            "상담내용요약": {
                "raw_text": row["상담내용"],
                "Embedding": []
            },
            "진행_과정_및_결과": row["진행 과정 및 결과"],
            "내담자_정보": {
                "거주지역": row["거주지역"],
                "국적": row["국적"],
                "체류자격": row["체류자격"],
                "추가정보": []  # LLM 추출 부분 빈 리스트로 초기화
            },
            "해결방법": "",  # LLM 추출 부분 빈 문자열로 초기화
            "metadata": {
                "상담유형": row["상담유형"],
                "상담일시": row["작성일"]
            }
        }
        documents.append(document)

    return documents


- 상담제목 쭉 뽑기, 상담내용요약 쭉 뽑기, -> 임베딩
- 내담자정보 -> 프롬프트 통해서 생성

## 2023년 이후 데이터, jsonl 변환하기

In [ ]:
data.head(2)

In [ ]:
# 작성일 열을 datetime으로 변환
data['작성일'] = pd.to_datetime(data['작성일'], format='%y-%m-%d')
from datetime import datetime

# 2023년 1월 1일 이후의 데이터만 필터링
cutoff_date = datetime(2023, 1, 1)
filtered_data = data[data['작성일'] >= cutoff_date]

# '작성일' 컬럼을 문자열로 변환
filtered_data['작성일'] = filtered_data['작성일'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# 데이터 변환
documents = transform_dataframe_to_documents(filtered_data)

In [ ]:
len(documents)

In [ ]:
type(documents)

저장

In [ ]:
# JSONL 형식으로 저장
with open('../data/consultation_documents_filtered2023.jsonl', 'w', encoding='utf-8') as f:
    for item in documents:  # documents는 이미 원하는 형식의 딕셔너리들의 리스트
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

확인

In [ ]:
# 저장된 JSONL 파일 확인
with open('../data/consultation_documents_filtered2023.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 2:  # 처음 두 줄만 확인
            print(f"Line {i+1}:")
            print(line.strip())
            print()

## 2022년 기준으로도 만들어놓기

In [ ]:
data.tail(2)

In [ ]:
# 작성일 열을 datetime으로 변환
data['작성일'] = pd.to_datetime(data['작성일'], format='%y-%m-%d')
from datetime import datetime

# 2022년 1월 1일 이후의 데이터만 필터링
cutoff_date = datetime(2022, 1, 1)
filtered_data = data[data['작성일'] >= cutoff_date]

# '작성일' 컬럼을 문자열로 변환
filtered_data['작성일'] = filtered_data['작성일'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# 데이터 변환
documents = transform_dataframe_to_documents(filtered_data)

In [ ]:
type(documents)

In [ ]:
for row in documents:
    print(row['상담제목'])

In [ ]:
len(documents)

In [ ]:
type(documents)

In [ ]:
def save_documents_to_jsonl(documents: List[Dict], file_path: str):
    """
    문서 리스트를 JSONL 파일로 저장

    Args:
        documents: 문서 딕셔너리 리스트
        file_path: 저장할 파일 경로
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in documents:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

In [ ]:
save_documents_to_jsonl(documents, '../data/consultation_documents_filtered2022.jsonl')

In [ ]:
import json
from typing import List, Dict
import pandas as pd

def validate_jsonl(file_path: str, show_sample: int = 2) -> tuple:
    """
    JSONL 파일을 검증하고 정보를 반환

    Args:
        file_path: JSONL 파일 경로
        show_sample: 출력할 샘플 개수

    Returns:
        tuple: (총 라인 수, 문서 리스트)
    """
    documents = []
    line_count = 0

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    doc = json.loads(line.strip())
                    documents.append(doc)
                    line_count += 1

                    # 샘플 데이터 출력
                    if line_count <= show_sample:
                        print(f"\n=== Document {line_count} ===")
                        print(json.dumps(doc, ensure_ascii=False, indent=2))
                        print("="*50)

                except json.JSONDecodeError as e:
                    print(f"Error parsing line {line_count + 1}: {e}")
                    continue

        print(f"\n총 문서 수: {line_count}")

        # 데이터 구조 검증
        if documents:
            print("\n문서 구조 검증:")
            required_fields = [
                "상담제목", "상담내용요약", "진행_과정_및_결과",
                "내담자_정보", "해결방법", "metadata"
            ]

            for field in required_fields:
                field_exists = all(field in doc for doc in documents)
                print(f"- {field}: {'✓' if field_exists else '✗'}")

        return line_count, documents

    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return 0, []
    except Exception as e:
        print(f"파일 읽기 오류: {e}")
        return 0, []

def compare_with_dataframe(documents: List[Dict], df: pd.DataFrame):
    """
    변환된 문서와 원본 데이터프레임 비교

    Args:
        documents: JSONL에서 읽은 문서 리스트
        df: 원본 데이터프레임
    """
    print(f"\n데이터 수 비교:")
    print(f"- 데이터프레임 행 수: {len(df)}")
    print(f"- JSONL 문서 수: {len(documents)}")

    if len(documents) == len(df):
        print("✓ 데이터 수가 일치합니다.")
    else:
        print("✗ 데이터 수가 일치하지 않습니다!")

# 사용 예시
if __name__ == "__main__":
    file_path = '/content/drive/MyDrive/HelloWorld/AI/data/consultation_documents_filtered2022.jsonl'

    # JSONL 파일 검증
    total_lines, documents = validate_jsonl(file_path)

    # 원본 데이터프레임이 있다면 비교
    # df = pd.read_csv('original_data.csv')  # 원본 데이터프레임 로드
    # compare_with_dataframe(documents, df)

    # 추가 정보 출력
    if documents:
        print("\n첫 번째 문서의 키:")
        first_doc = documents[0]
        for key, value in first_doc.items():
            if isinstance(value, dict):
                print(f"- {key}: {list(value.keys())}")
            else:
                print(f"- {key}: {type(value).__name__}")

In [ ]:
# JSONL 형식으로 저장
with open('/content/drive/MyDrive/HelloWorld/AI/data/consultation_documents_filtered2022.jsonl', 'w', encoding='utf-8') as f:
    for item in documents:  # documents는 이미 원하는 형식의 딕셔너리들의 리스트
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

확인

In [ ]:
# 저장된 JSONL 파일 확인
with open('/content/drive/MyDrive/HelloWorld/AI/data/consultation_documents_filtered2022.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 2:  # 처음 두 줄만 확인
            print(f"Line {i+1}:")
            print(line.strip())
            print()

# Embedding 추가 구축

In [5]:
filtered_data.head()

,제목,작성일,상담유형,거주지역,국적,체류자격,상담내용,진행 과정 및 결과,관련법령 및 정보,평가 및 의의,상담지원단체,작성자,url
0,중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원,2023-11-27,체류자격,여주시,우즈베키스탄,E-9,중도 퇴직하는 외국인 근로자의 추가 납부세액 사례,2023-09-18\n- 고용노동부에서 외국인근로자의 2020년 종합소득세 체납을 ...,,,여주시외국인복지센터,이수현,https://gmhr.or.kr/case/1703?page=1
1,국민연금 반환일시금 신청 지원,2023-11-27,기타,남양주시,필리핀,미등록,2023년 8월 중순 미등록체류 상황에서 본국으로 강제출국 당한 필리핀 국적의 이주...,11월 2일\n1. 노동자로부터 우편으로 관련서류 수령.\n 1) 외국인등록증 사본...,,,남양주시외국인복지센터,김성수,https://gmhr.or.kr/case/1702?page=1
2,외국인아동 교육권 보장을 위한 체류자격 부여자(G-1-85) 외국인등록 절차 지원,2023-11-27,체류자격,남양주시,필리핀,미등록,법무부 장기 미등록체류 이주아동 구제대책으로 23년 4월에 접수한 필리핀 국적의 아...,11월 1일\n1. 양주출입국 사범과 방문.\n2. 담당자로부터 유전자 검사 결과 ...,,,남양주시외국인복지센터,김성수,https://gmhr.or.kr/case/1701?page=1
3,단체행동을 통해 체불임금 해결한 사례,2023-11-27,임금체불,안산시,필리핀,E-9,"사례자 안산시 한 직장에서 근무하는데, 회사에서 잔업수당 등을 상습적으로 속이고 있...",8월 14일\n(전화상담) 회사에서 급여계산을 잘못해서 주고 야간 수당을 안주고 있...,,,글로벌미션센터,최경식,https://gmhr.or.kr/case/1700?page=1
4,취업가능기간이 짧게 남아 취업이 어려운 노동자 구직활동 지원,2023-11-27,고용허가,시흥시,필리핀,E-9,상기 사례자는 한국 체류기간(노동가능기간)이 1년 4개월로 비교적 짧게 남아서 면접...,4월 2일\n(내방상담) 현재 다니는 2번째 회사에도 일이 없어서 사업장변경 신청하...,,,글로벌미션센터,최경식,https://gmhr.or.kr/case/1699?page=1


In [6]:
from utils import BatchClient
batch_client = BatchClient(config_path="../configs/config.json")

## config loaded ##
## client loaded ##


In [7]:
FILE_YEAR = "2023"

In [8]:
filtered_data.columns

Index(['제목', '작성일', '상담유형', '거주지역', '국적', '체류자격', '상담내용', '진행 과정 및 결과',
       '관련법령 및 정보', '평가 및 의의', '상담지원단체', '작성자', 'url'],
      dtype='object')

In [11]:
# embedding batch 처리
import time

metadata = batch_client.make_generate(df=filtered_data, column_name="해결방법", 
                                      prompt_path="../prompts/generate_prompt.json",
                                      batch_jsonl_path=f"../data/batch_jsonl/input/consultation_documents_{FILE_YEAR}_batch.jsonl",
                                      meta_path=f"../data/meta_csv/consultation_documents_{FILE_YEAR}_meta.csv")
input_file_id =  metadata['input_file_id']
print('## input file id : ', input_file_id)

# while True:
#     status = BatchClient.get_batch_status(input_file_id)
#     print('## current status : ', status.status)

#     if status.status == "completed":
#         output_file_id = status.output_file_id
#         print(f'## batch completed!\noutput file id : {output_file_id}')
#         break

#     else:
#         print('## Wait untill completed...')
#         time.sleep(60)

# BatchClient.get_batch_answer(output_file_id=input_file_id, result_path=f"../data/batch_jsonl/output/consultation_documents_{FILE_YEAR}_batch.jsonl")


TypeError: expected str, bytes or os.PathLike object, not NoneType